In [6]:
import glob
import pandas as pd
import numpy as np
import os

In [7]:
result_files = glob.glob("result/" + '*.txt')

In [8]:
result_files.sort(key=os.path.getmtime)

In [9]:
# load sim data
sim_file_path                              = "C:/Users/eornr/Desktop/source_crx/200331_point_gen/01_point_gen_in_STL/06_gen_point_data_proto/sim_pose/"
sim_file_names                             = ["left_femur_sim", "left_tibia_sim", "right_femur_sim", "right_tibia_sim"]

left_femur_position_pose_data_list         = np.load(sim_file_path + sim_file_names[0] + '.npy')
left_tibia_position_pose_data_list         = np.load(sim_file_path + sim_file_names[1] + '.npy')
right_femur_position_pose_data_list         = np.load(sim_file_path + sim_file_names[2] + '.npy')
right_tibia_position_pose_data_list         = np.load(sim_file_path + sim_file_names[3] + '.npy')

## gen positioning mtx
# FL, TL, FR, TR
trns_mtx_list                              = [left_femur_position_pose_data_list,
                                              left_tibia_position_pose_data_list,
                                              right_femur_position_pose_data_list,
                                              right_tibia_position_pose_data_list]

transform_data_euler_degree_rot            = []

sim_files_count                            = [0, 0, 0, 0]

ans_mtx                                    = []

for file_name in result_files:
    if("F" in file_name and "L" in file_name):
        sim_files_count[0] = sim_files_count[0] + 1
    elif("T" in file_name and "L" in file_name):
        sim_files_count[1] = sim_files_count[1] + 1
    elif("F" in file_name and "R" in file_name):
        sim_files_count[2] = sim_files_count[2] + 1
    elif("T" in file_name and "R" in file_name):
        sim_files_count[3] = sim_files_count[3] + 1

for i in range(len(sim_files_count)):
    if (sim_files_count[i] == 216):
        transform_data_euler_degree_rot.append(trns_mtx_list[i])
        
transform_data_euler_degree_rot_all = []

for i in range(len(transform_data_euler_degree_rot)):
    for j in range(len(transform_data_euler_degree_rot[i])):
        transform_data_euler_degree_rot_all.append(transform_data_euler_degree_rot[i][j])

In [10]:
data_step_log = []
data_Euler = []

for i in range(len(result_files)):
    with open(result_files[i], 'r') as file:
        data_log = []
        data_value = []
        
        data_log.append(result_files[i][14:18] + result_files[i][-8:-4])
        
        for j in range(len(transform_data_euler_degree_rot_all[i])):
            data_log.append(str(transform_data_euler_degree_rot_all[i][j]))
        
        for line in file:
            tmp_str = ""
            if ":" in line:
                _line = line.split(" : ")
                if "\n" in _line[1]:
                    tmp_str = _line[1].split("\n")[0]
                if "\x00" in _line[1]:
                    tmp_str = _line[1].split("\x00")[0]
                data_log.append(tmp_str)
            elif "--" in line:
                pass
            elif "  \n" == line:
                pass
            else:
                _line = line.split("\n")
                data_value.append(_line[0])
                
        data_step_log.append(data_log)
        data_Euler.append(data_value)

In [11]:
data_frame_colume_head = ["data name", 
                          "sim. pose Tx", "sim. pose Ty", "sim. pose Tz", "sim. pose Rx", "sim. pose Rx", "sim. pose Rx",
                          "coarse iter", "coarse RMSE", "fine iter", "fine RMSE", "fine RMSE all pts", 
                          "error Tx", "error Ty", "error Tz", "error Rx", "error Ry", "error Rz"]

data_frame_list = []

data_frame_list.append(data_frame_colume_head)

for i in range(len(data_step_log)):
    data_frame_list.append(data_step_log[i] + data_Euler[i])

In [12]:
df = pd.DataFrame(data_frame_list) 

In [13]:
df.to_excel('result/' + 'output.xlsx', index=False)